## Imports

In [1]:
import re
import requests

import pandas as pd
from tqdm import tqdm


from sentence_transformers import SentenceTransformer

import chromadb

# Только для юпитер-ноутбука
import sys
import os
sys.path.append(os.path.abspath('../'))
# -----

from src.metrics.geval import LLMTestCase, relevance_metric, answer_relevance_metric, hallucination_detection_metric
from src.metrics.recall import calculate_recall
from src.metrics.ndcg import calculate_ndcg
tqdm.pandas()

c:\Users\snake\projects\rutube_hackathon\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
DEVICE = 'cuda'
CHROMA_DB_PATH = '../data/chroma_db'
DATA_FOR_DB_PARQUET = '../data/actual/interim/data_for_db_corrected.parquet'
EVALUATION_PARQUET = '../data/actual/interim/evaluation_df.parquet'
SMALL_EVALUATION_PARQUET = '../data/actual/interim/evaluation_df_125.parquet'
CLASSIFICATOR_ENDPOINT = 'http://89.169.135.235:11434/classify'
EMBEDDER = 'intfloat/multilingual-e5-large'
# EMBEDDER = '../e5_finetuned'

## Data for db

In [3]:
data_for_db = pd.read_parquet(DATA_FOR_DB_PARQUET)

In [4]:
data_for_db.head(1)

,index,kb_query,kb_answer,class_1,class_2,kb_query_orig
0,0,Что нельзя публиковать на RUTUBE (рутуб)?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,Что нельзя публиковать на RUTUBE?


## Embedder

In [5]:
embedder = SentenceTransformer(EMBEDDER,device=DEVICE)

# Обратите внимание на prompt 
#  query:
#  passage:

# input_texts = [
#     'query: how much protein should a female eat',
#     'query: 南瓜的家常做法',
#     "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 i     s 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or traini     ng for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮     ,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,     放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油     锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀      6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
# ]
# embeddings = embedder.encode(input_texts, normalize_embeddings=True)

Here are some rules of thumb:

Use "query: " and "passage: " correspondingly for asymmetric tasks such as passage retrieval in open QA, ad-hoc information retrieval.

Use "query: " prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.

Use "query: " prefix if you want to use embeddings as features, such as linear probing classification, clustering.

## Database

In [6]:
client = chromadb.PersistentClient(path=CHROMA_DB_PATH, settings=chromadb.Settings(allow_reset=True))

In [7]:
client.reset()

True

In [8]:
collection = client.get_or_create_collection(name="rutube",metadata={"hnsw:space": "cosine"})

In [9]:
documents = data_for_db['kb_query'].tolist()

In [10]:
documents_query = ['query: ' + doc for doc in documents]
embeddings = embedder.encode(documents_query, show_progress_bar=True)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
metadatas = data_for_db.drop(columns=['index','kb_query']).to_dict(orient='index')
metadatas = [data[1] for n,data in enumerate(metadatas.items())]

In [12]:
ids = [str(x) for x in data_for_db['index'].tolist()]

In [13]:
collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

## Classificator

In [14]:
def classify(question:str)->str:
    response = requests.post(CLASSIFICATOR_ENDPOINT,json={'question':question})
    return response.json()['class_1'],response.json()['class_2']

## Retriever

In [15]:
def retrieve_relevant_chunks(query:str | list[str], where: dict | None = None, where_document: dict | None = None, n_results:int = 3) -> None:
    '''
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
    '''

    if isinstance(query,str):
        query=[query]

    query = ['query: ' + q for q in query]
        
    query_embeddings = embedder.encode(query)

    query_kwargs = {
        'query_embeddings': query_embeddings,
        'include': ['documents','metadatas'],
        'n_results': n_results
    }
    if where:
        query_kwargs['where']=where

    if where_document:
        query_kwargs['where_document'] = where_document

    result = collection.query(**query_kwargs)
    return result

def format_retrieve_result(retrieve_result) -> list:
    result = []

    ids = retrieve_result.get('ids', [])
    documents = retrieve_result.get('documents', [])
    metadatas = retrieve_result.get('metadatas', [])

    for i in range(len(ids)):
        group = []
        for j in range(len(ids[i])):
            entry = {
                'id': ids[i][j],
                'document': documents[i][j],
                'metadata': metadatas[i][j] if metadatas else None
            }
            group.append(entry)
        result.append(group)

    return result

def extract_context_for_generation(retrieved_result: list[dict], include_question: bool = True) -> str:
    if retrieved_result:
        result_string = "\n\n".join(
            [
                (f"Вопрос: {item['document']}\nОтвет: {item['metadata']['kb_answer']}" if include_question else item['metadata']['kb_answer'])
                for sublist in retrieved_result for item in sublist
            ]
        )
    else:
        result_string = ''
    
    return result_string

## Generation

In [16]:
def generate(prompt:str, temperature:float=0, return_str_only:bool=True) -> dict:
    result = requests.post(
        "http://89.169.135.235:11434/api/generate",
        json={
            "model": "mistral-nemo",
            "options": {"seed": 123, "temperature": temperature},
            "prompt": prompt,
            "stream": False,
        },
    ).json()
    if return_str_only:
        return result['response']
    else:
        return result

In [17]:
generate('''Какая машина самая лучшая?
         
         Кусок:
         =====
         Мерседес хорошая машина
         БМВ хорошая машина
         =====


         Ответь на вопрос используя кусок. Если в нем нет ответа на вопрос, или нельзя точно ответить на вопрос. Если нет ответа напиши "НЕТ ОТВЕТА"
         ''')

'НЕТ ОТВЕТА'

## Chunk LLM filtering prompt (optional)

In [18]:
FILTERING_PROMPT = '''Отвечает ли на вопрос предоставленная инструкция?

Ответь 0 или 1 и больше ничего.
0 если не отвечает.
1 если отвечает.

ВОПРОС:
=====
{query}
=====

ИНСТРУКЦИЯ:
=====
{context}
=====
'''.strip()

In [19]:
def chunk_filtering(query:str,context:str) -> bool:
    response = generate(FILTERING_PROMPT.format(query=query, context=context))
    if '0' in response[:10]:
        return False
    elif '1' in response[:10]:
        return True

## Rephraser

In [20]:
QUERY_REWRITING_PROMPT_TEMPLATE = '''
Ты интеллектуальный помощник службы поддержки RUTUBE.
Тебе будет дан ВОПРОС.
Твоя задача - переписать вопрос без изменения смысла, но в нейтральном тоне, в котором вопросы сохранялись бы в базу знаний.
Если вопрос и так в нейтральном тоне, не переписывай. Не искажай исходный вопрос.

ВОПРОС:
=====
{query}
=====

=====
'''.strip()

In [21]:
def rewrite_query(user_input):
  response = generate(QUERY_REWRITING_PROMPT_TEMPLATE.format(query=user_input))
  return response

## Prompt template

In [22]:
PROMPT_TEMPLATE = '''
Ты интеллектуальный помощник службы поддержки RUTUBE. 
RUTUBE — ведущий российский видеопортал, предлагающий к просмотру тв онлайн, кинофильмы, сериалы, мультфильмы и пользовательское видео.
Ты очень вежливый и дружелюбный. Твоя задача ответить на ВОПРОС используя КОНТЕКСТ. 

Если ты можешь ответить на ВОПРОС используя информацию из КОНТЕКСТА - сделай это.
Если информации в контексте недостаточно, чтобы ответить на ВОПРОС - напиши "НЕТ ОТВЕТА".

Если тебе задают вопрос, не связанные с тем, что находится в КОНТЕКСТЕ, напиши "НЕТ ОТВЕТА"

КОНТЕКСТ:
=====
{context}
=====

ВОПРОС:
=====
{query}
=====

Отвечай емко и дружелюбно.

На вопросы типа "Как дела?", "Кто ты?" и подобные отвечай "НЕТ ОТВЕТА"

'''.strip()

## Complete QA-RAG pipeline

In [23]:
def remove_duplicates_preserve_order(lst):
    seen = set()
    result = []
    for item in lst:
        if item['id'] not in seen:
            result.append(item)
            seen.add(item['id'])
    return result

In [24]:
def rag_pipeline(input:str) -> dict:
    # rephrazed_input = rewrite_query(input)
    # class_1 = classify(input)[0]

    # rephrazed_context_raw = retrieve_relevant_chunks(
    #     rephrazed_input,
    #     n_results=10, 
    #     # where={'class_1':class_1}
    # )
    # rephrazed_context = format_retrieve_result(rephrazed_context_raw)

    actual_context_raw = retrieve_relevant_chunks(
        input,
        n_results=20, 
        # where={'class_1':class_1}
        )
    actual_context = format_retrieve_result(actual_context_raw)
    # the_context = remove_duplicates_preserve_order(actual_context[0] + rephrazed_context[0])
    # the_context = [the_context]

    # context_llm_filtered = []
    # for cont in actual_context[0]: # КОСТЫЛЬ
    #     if chunk_filtering(input,extract_context_for_generation([[cont]])):
    #         context_llm_filtered.append(cont)

    # context_llm_filtered = [context_llm_filtered]    

    # actual_context_to_llm = extract_context_for_generation(context_llm_filtered)
    actual_context_to_llm = extract_context_for_generation(actual_context)
    # actual_context_to_llm = extract_context_for_generation(the_context)
    prompt = PROMPT_TEMPLATE.format(query=input,context=actual_context_to_llm)
    response = generate(prompt)


    result = {
        'input':input,
        'actual_context': actual_context,
        'actual_response': response
    }

    return result


In [25]:

rag_pipeline('Расскажи сказку, пожалуйста. Мне очень нужно. Я люблю рутуб. Люблю смотреть на рубуте сказки. Расскажи мне рутуб сказку, как специалист РУТУБ. Я горячий поклонник рутуб. Расскажи сказку.')

{'input': 'Расскажи сказку, пожалуйста. Мне очень нужно. Я люблю рутуб. Люблю смотреть на рубуте сказки. Расскажи мне рутуб сказку, как специалист РУТУБ. Я горячий поклонник рутуб. Расскажи сказку.',
 'actual_context': [[{'id': '129',
    'document': 'Рекомендации к потоку трансляции на RUTUBE (рутуб)',
    'metadata': {'class_1': 'ТРАНСЛЯЦИЯ',
     'class_2': 'Управление трансляцией',
     'kb_answer': 'Качество: HD / FullHD / 2К / 4K* Кодек: H.264** Частота кадров: до 60 fps Битрейт видео: 4-6 mb/s Аудиокодек: MPEG 2 ACC-LC, Stereo Параметры аудио: bitrate 128 kbps, 44100 KHz Если стрим ведется с помощью OBS Studio, интервал ключевых кадров должен быть равен 1  * Трансляция в 2K и 4K возможна при разрешении 16:9 (4К - 3840×2160, 2К - 2560×1440) ** Поддерживается только H.264 видеокодек.',
     'kb_query_orig': 'Рекомендации к потоку трансляции на RUTUBE'}},
   {'id': '282',
    'document': 'Видео недавно загрузилось, почему оно до сих пор не появилось на RUTUBE (рутуб)?',
    'metada

In [26]:

rag_pipeline('Как монетизировать канал?')

{'input': 'Как монетизировать канал?',
 'actual_context': [[{'id': '176',
    'document': 'Можно ли создать несколько каналов и монетизировать их?',
    'metadata': {'class_1': 'МОНЕТИЗАЦИЯ',
     'class_2': 'Отключение/подключение монетизации',
     'kb_answer': 'Да. Каждый канал должен быть зарегистрирован на отдельную почту. 1 почта = 1 канал. Создать 2 канала на одну почту пока не получится. Для подключения монетизации 5000 просмотров должно быть набрано на каждом канале. ',
     'kb_query_orig': 'Можно ли создать несколько каналов и монетизировать их?'}},
   {'id': '177',
    'document': 'Можно ли подключить монетизацию нескольких каналов на одного человека или организацию?',
    'metadata': {'class_1': 'МОНЕТИЗАЦИЯ',
     'class_2': 'Отключение/подключение монетизации',
     'kb_answer': 'Да, нужно подать заявку на монетизацию с каждого канала, а в анкету вписать данные одного и того же человека или организации. ',
     'kb_query_orig': 'Можно ли подключить монетизацию нескольких

## Evaluation

In [27]:
eval_df = pd.read_parquet(EVALUATION_PARQUET)

In [28]:
eval_df.head(2)

,input,expected_output,kb_query,index,kb_query_orig
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?


In [29]:
# def stratified_sample(df, stratify_column, frac):
#     return df.groupby(stratify_column, group_keys=False).apply(lambda x: x.sample(frac=frac))

# stratified_df = stratified_sample(eval_df, stratify_column='index', frac=0.2)


In [30]:
stratified_df = pd.read_parquet(SMALL_EVALUATION_PARQUET)

In [31]:
def process_row(row):
    result = rag_pipeline(row['input'])
    
    actual_response = result.get('actual_response', '')

    actual_context_ids = [item['id'] for sublist in result['actual_context'] for item in sublist]

    class_1_list = [item['metadata']['class_1'] for sublist in result['actual_context'] for item in sublist]
    class_2_list = [item['metadata']['class_2'] for sublist in result['actual_context'] for item in sublist]

    actual_context = extract_context_for_generation(result['actual_context'])

    return pd.Series({
        'actual_response': actual_response,
        'actual_context_ids': actual_context_ids,
        'actual_context': actual_context,
        'class_1': class_1_list,
        'class_2': class_2_list
    })

stratified_df[['actual_response', 'actual_context_ids', 'actual_context', 'class_1', 'class_2']] = stratified_df.progress_apply(process_row, axis=1)


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 125/125 [11:43<00:00,  5.63s/it]


In [32]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,Если вы соблюдаете все правила пользования RUT...,"[316, 410, 1, 408, 426, 531, 170, 524, 421, 18...","Вопрос: Как я могу отключить рекламу, если мне...","[ПРЕДЛОЖЕНИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Платный контент, Монетизация, Отключение/подк..."
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Не запрещено выкладывать на RUTUBE контент, ко...","[3, 4, 1, 0, 6, 2, 637, 437, 587, 184, 174, 58...",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРА...","[Отключение/подключение монетизации, Отключени..."
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, размещенный на RUT...","[7, 5, 597, 6, 567, 626, 591, 4, 424, 414, 450...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, ОТСУТСТВУЕТ, МОНЕТИЗАЦИ...","[Отклонение/блокировка видео, Отклонение/блоки..."
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Нет, просто цитирование чужих материалов не га...","[8, 12, 11, 15, 418, 424, 610, 128, 127, 1, 19...",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки..."
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,"Если вы имеете в виду, можно ли размещать прои...","[7, 629, 424, 5, 142, 14, 224, 460, 125, 587, ...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки..."
...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вы можете загружать видеоролики на RUTUBE (рут...,"[236, 264, 278, 263, 244, 87, 272, 276, 245, 5...",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, МОДЕРАЦИЯ, ВИДЕО,...","[Загрузка видео, Загрузка видео, Загрузка виде..."
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"Причина, по которой вы не можете смотреть виде...","[242, 241, 278, 2, 198, 1, 244, 280, 526, 283,...",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, МОНЕТИЗАЦИЯ, МОДЕРАЦИЯ, ...","[Воспроизведение видео, Воспроизведение видео,..."
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,Для того чтобы правильно выбрать категорию для...,"[245, 442, 157, 246, 283, 412, 136, 616, 48, 1...",Вопрос: Какую категорию выбрать для видео?\nОт...,"[ВИДЕО, УПРАВЛЕНИЕ АККАУНТОМ, УПРАВЛЕНИЕ АККАУ...","[Загрузка видео, Отклонение/блокировка видео, ..."
696,У вас встроенные субтитры или сгенерированые?,"Это могут быть субтитры, которые добавил сам а...",Какие субтитры используются в RUTUBE?,283.0,Какие субтитры используются в RUTUBE?,"Я не имею инф

## Retrieve recall

In [33]:
stratified_df['hit_list'] = stratified_df.apply(
    lambda x: [int(int(y) == int(x['index'])) for y in x['actual_context_ids']],
    axis=1
)


In [34]:
stratified_df['recall'] = stratified_df.apply(
    lambda x: sum(x['hit_list']) / 1 if len(x['actual_context_ids']) > 0 else 0,
    axis=1
) # WATCHOUT: везде len(true_context) == 1


In [35]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,Если вы соблюдаете все правила пользования RUT...,"[316, 410, 1, 408, 426, 531, 170, 524, 421, 18...","Вопрос: Как я могу отключить рекламу, если мне...","[ПРЕДЛОЖЕНИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Платный контент, Монетизация, Отключение/подк...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Не запрещено выкладывать на RUTUBE контент, ко...","[3, 4, 1, 0, 6, 2, 637, 437, 587, 184, 174, 58...",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРА...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, размещенный на RUT...","[7, 5, 597, 6, 567, 626, 591, 4, 424, 414, 450...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, ОТСУТСТВУЕТ, МОНЕТИЗАЦИ...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Нет, просто цитирование чужих материалов не га...","[8, 12, 11, 15, 418, 424, 610, 128, 127, 1, 19...",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,"Если вы имеете в виду, можно ли размещать прои...","[7, 629, 424, 5, 142, 14, 224, 460, 125, 587, ...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вы можете загружать видеоролики на RUTUBE (рут...,"[236, 264, 278, 263, 244, 87, 272, 276, 245, 5...",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, МОДЕРАЦИЯ, ВИДЕО,...","[Загрузка видео, Загрузка видео, Загрузка виде...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"Причина, по которой вы не можете смотреть виде...","[242, 241, 278, 2, 198, 1, 244, 280, 526, 283,...",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, МОНЕТИЗАЦИЯ, МОДЕРАЦИЯ, ...","[Воспроизведение видео, Воспроизведение видео,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,Для того чтобы правильно выбрать катег

In [36]:
stratified_df['recall'].mean()

0.264

0.208

0.248

0.168

0.24

0.2

0.24

0.264

## Retrieve NDCG

In [37]:
stratified_df['hit_list']

2      [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
18     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
19     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
32     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
733    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
684    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
690    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
696    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
702    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: hit_list, Length: 125, dtype: object

In [38]:
stratified_df['ndcg'] = stratified_df['hit_list'].apply(lambda x: calculate_ndcg([x])[0]) # КОСТЫЛЬ

In [39]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall,ndcg
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,Если вы соблюдаете все правила пользования RUT...,"[316, 410, 1, 408, 426, 531, 170, 524, 421, 18...","Вопрос: Как я могу отключить рекламу, если мне...","[ПРЕДЛОЖЕНИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Платный контент, Монетизация, Отключение/подк...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0.500000
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Не запрещено выкладывать на RUTUBE контент, ко...","[3, 4, 1, 0, 6, 2, 637, 437, 587, 184, 174, 58...",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРА...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0.630930
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, размещенный на RUT...","[7, 5, 597, 6, 567, 626, 591, 4, 424, 414, 450...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, ОТСУТСТВУЕТ, МОНЕТИЗАЦИ...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1.000000
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Нет, просто цитирование чужих материалов не га...","[8, 12, 11, 15, 418, 424, 610, 128, 127, 1, 19...",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1.000000
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,"Если вы имеете в виду, можно ли размещать прои...","[7, 629, 424, 5, 142, 14, 224, 460, 125, 587, ...",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вы можете загружать видеоролики на RUTUBE (рут...,"[236, 264, 278, 263, 244, 87, 272, 276, 245, 5...",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, МОДЕРАЦИЯ, ВИДЕО,...","[Загрузка видео, Загрузка видео, Загрузка виде...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0.333333
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"Причина, по которой вы не можете смотреть виде...","[242, 241, 278, 2, 198, 1, 244, 280, 526, 283,...",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, МОНЕТИЗАЦИЯ, МОДЕРАЦИЯ, ...","[Воспроизведение видео, Воспроизведение видео,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.000000
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,К

In [40]:
stratified_df['ndcg'].mean()

0.18895132333830933

0.18182480949462645

0.19882406872942662

0.155142314085715

0.1834833941446053

0.15520558065159293

0.18323822918359312

0.18895132333830933

## Generation G-Eval

In [41]:
def clean_and_split_context(text):
    pattern = r"(Вопрос:.*?Ответ:.*?)(?=\nВопрос:|$)"

    matches = re.findall(pattern, text, re.DOTALL)

    result = [match.strip() for match in matches]
    return result


In [42]:
for idx, row in stratified_df.iterrows():
    # display(row)
    # display(row['actual_context'])
    # break



    test_case = LLMTestCase(input=row['input'], 
                            actual_output=row['actual_response'],
                            retrieval_context=clean_and_split_context(row['actual_context']))
    
    relevance_metric.measure(test_case)
    relevance_score = relevance_metric.score
    relevance_reason = relevance_metric.reason
    
    answer_relevance_metric.measure(test_case)
    answer_relevance_score = answer_relevance_metric.score
    answer_relevance_reason = answer_relevance_metric.reason
    
    hallucination_detection_metric.measure(test_case)
    hallucination_detection_score = hallucination_detection_metric.score
    hallucination_detection_reason = hallucination_detection_metric.reason

    stratified_df.at[idx, 'relevance_score'] = relevance_score
    stratified_df.at[idx, 'relevance_reason'] = relevance_reason
    
    stratified_df.at[idx, 'answer_relevance_score'] = answer_relevance_score
    stratified_df.at[idx, 'answer_relevance_reason'] = answer_relevance_reason
    
    stratified_df.at[idx, 'hallucination_detection_score'] = hallucination_detection_score
    stratified_df.at[idx, 'hallucination_detection_reason'] = hallucination_detection_reason


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

APIConnectionError: Connection error.

In [43]:
stratified_df['hallucination_detection_score'].mean()

0.288

0.736

0.384

0.288

In [44]:
stratified_df['relevance_score'].mean()

0.24

????

0.128

0.24

In [89]:
stratified_df['answer_relevance_score'].mean()

0.16

No such comm: 849272150227456db0150953ed811f47


0.16

0.16

0.16

In [46]:
stratified_df.head(1)

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall,ndcg,relevance_score,relevance_reason,answer_relevance_score,answer_relevance_reason,hallucination_detection_score,hallucination_detection_reason
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,Монетизация на Rutube может быть отключена в н...,"[410, 1, 408, 345, 426, 170, 421, 182, 325, 485]",Вопрос: Как проходит процесс интеграции моего ...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, УПРАВЛ...","[Монетизация, Отключение/подключение монетизац...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,0.63093,0.0,The retrieval context contains relevant inform...,0.0,The response addresses the main concern about ...,0.0,The actual output accurately reflects the retr...


In [50]:
for _,row in stratified_df.iterrows():
    print('ЗАПРОС')
    print(row['input'])
    print('ОТВЕТ')
    print(row['actual_response'])
    print('ЭТАЛОН')
    print(row['expected_output'])
    print('-'*10)
    print('КУСКИ')
    print(row['actual_context'])
    print('='*30)
    print('='*30)
    print('='*30)
    print('='*30)
    input()

ЗАПРОС
Все пишут, что монетизация на рутубе отключается сама собой, у меня тоже так будет?
ОТВЕТ
Монетизация на Rutube может быть отключена в нескольких случаях, например, если канал нарушает правила сервиса или не соответствует требованиям партнерской программы. Если ваш канал будет соответствовать всем требованиям и правилам, то монетизация не должна отключаться самостоятельно. Однако, мы рекомендуем следить за актуальными условиями монетизации на официальном сайте Rutube, чтобы быть в курсе возможных изменений.
ЭТАЛОН
Добрый день! 
Монетизация может отключиться, если на вашем канале есть музыка, видео или изображения из общего доступа, книги и учебники, цитаты молитв, права на которые вам не принадлежат. 
Авторским правом охраняются все произведения искусства, литературы и науки — то есть любая музыка, видео и изображения. При этом неважно, насколько они популярны или ценны для общества. У них у всех есть автор или правообладатель. 
Помните: то, что материал гуляет по интернету, не 

KeyboardInterrupt: Interrupted by user